### 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

아래 코드의 주석을 풀고 발급받은 API KEY 를 설정합니다.


In [1]:
import os

# TAVILY API KEY를 기입합니다.
# os.environ["TAVILY_API_KEY"] = "TAVILY API KEY 입력"

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=3)

### Agent 가 사용할 도구 목록 정의


In [4]:
# tools 리스트에 search 도구를 추가합니다.
tools = [search]

## Agent with smaller model

- reference: https://github.com/langchain-ai/langchain/blob/master/templates/neo4j-semantic-ollama/neo4j_semantic_ollama/agent.py


In [5]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage

from langchain.pydantic_v1 import BaseModel, Field


local_llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
    temperature=0,
)

tools = [search]

llm_with_tools = local_llm.bind_tools(tools)

In [6]:
from typing import Tuple, List

chat_model_with_stop = local_llm.bind(stop=["Observation", "\nObservation", "\n관측"])

# Inspiration taken from hub.pull("hwchase17/react-json")
system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action... 
(this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            system_message,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [7]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"]),
        "chat_history": lambda x: (
            _format_chat_history(x["chat_history"]) if x.get(
                "chat_history") else []
        ),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
).with_types(input_type=AgentInput)

In [8]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = agent_executor.invoke(
    {
        "input": "판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.",
        "chat_history": [],
    }
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
<|im_start|>system
Perform the task to the best of your ability.<|im_end|>
<|im_start|>user
Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query., args: {'query': {'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: ['tavily_search_results_json']
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an e

In [9]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolExecutor
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnableParallel


def format_search_result(search_result):
    return " ".join([r["content"] for r in search_result])


tool_executor = ToolExecutor(tools)

In [10]:
search_agent = (
    agent_executor
    | itemgetter("output")
    | ReActJsonSingleInputOutputParser()
    | tool_executor
    | format_search_result
)

search_agent_parallel = RunnableParallel(
    context=search_agent, question=itemgetter("input")
)

In [11]:
search_agent_parallel.invoke(
    {
        "input": "판교 몽중헌 전화번호 웹 검색하여 알려줘.",
        "chat_history": [],
    }
)



> Entering new AgentExecutor chain...
<|im_start|>system
Perform the task to the best of your ability.<|im_end|>
<|im_start|>user
Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query., args: {'query': {'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: ['tavily_search_results_json']
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an e

{'context': "중식 | 판교 몽중헌 판교점 ... - 판교 테크원 타워 건물 내 지하 주차장을 이용 부탁드리며 , 2시간 이용가능 합니다. ... 전화번호 031-601-7574. 잘못된 정보가 있나요? 수정이 필요하거나 폐점한 매장이라면 알려주세요! 저장 1,225. 예약하기 ... *2인 이상 주문 시 가능합니다.\n냉채류\n탕류\n잡품류\n바닷가재·새우류\n쇠고기류\n돼지고기류\n닭고기류\n구이류\n채소·두부류\n면류\n밥류\nSteamed Dim Sum\n조리에 사용되는 닭육수용 닭은 국내산만 사용합니다.\n Chef-Recommended\nSpicy Dish\nDim Sum Special Course\nLUNCH - 75,000 (1人)\nDINNER - 95,000 (1人)\n조리에 사용되는 닭육수용 닭은 국내산만 사용합니다.\n 할인 제공 종료\n기프트 카드\n몽중헌 브랜드 전용 기프트 카드\nChef ’s Special Course / All Day\n180,000 (1人)\n조리에 사용되는 닭육수용 닭은 국내산만 사용합니다.\n 매장정보\n대중교통\n제휴 상품권\nCJ상품권, CJ외식전용상품권 *지류 상품권에 한해 적용 가능\n제휴 카드\n현대카드 블랙 / 퍼플 / 레드 / 그린 / 핑크 / 대한항공 더퍼스트 카드의 ‘클럽 고메 서비스’ 운영 종료로, 2024년 1월 1일부터 몽중헌 전 매장 10% 현장 Chef-Recommended\nSpicy Dish\nFried Dim Sum\n조리에 사용되는 닭육수용 닭은 국내산만 사용합니다.\n 📞 전화번호: 031-601-7574. 🌐 홈페이지: 몽중헌 공식 홈페이지 오늘은 가족모임 장소로 추천하는 분당 중식당 몽중헌을 소개해봅니다. 저는 가족모임이 있어서 룸이있는 식당을 찾다가 판교에 몽중헌이 있는걸 알고 예약을 한달전에 진행했어요 ㅋㅋ\n\u200b\n메인 메뉴판 보실게요\n\u200b\n\u200b\n사진을 찍다가 지쳐하니까\n앞에 일행이 메뉴판을 들어줬습니다\nㅋㅋㅋㅋㅋ\n민망..\n

In [12]:
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI Assistant. You must answer the question based on the context.\n#Context: {context}",
        ),
        ("user", "{question}"),
    ]
)

eeve = ChatOllama(
    model="EEVE-Korean-10.8B:long",
    temperature=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

chain = (
    RunnableParallel(question=itemgetter("question"),
                     context=itemgetter("context"))
    | prompt
    | eeve
    | StrOutputParser()
)

In [13]:
final_chain = search_agent_parallel | chain

In [14]:
final_chain.invoke(
    {
        "input": "판교 몽중헌 전화번호 검색하여 알려주세요",
        "chat_history": [],
    }
)



> Entering new AgentExecutor chain...
<|im_start|>system
Perform the task to the best of your ability.<|im_end|>
<|im_start|>user
Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query., args: {'query': {'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: ['tavily_search_results_json']
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an e

'판교 몽중헌의 전화번호는 다음과 같습니다: 031-601-7574.'